In [ ]:
from random import randrange
from math import pi
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.circuit.library.standard_gates.x import XGate, MCXGate
import math
from qiskit import transpile
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime.fake_provider.backends.cairo import FakeCairoV2
from qiskit.visualization import plot_circuit_layout
from qiskit.visualization import plot_histogram

In [ ]:
p=17  
n=4
K=[4, 8, 12, 6]
key =""
for cc in range(n):
    key+="0"

In [ ]:
def hadamardForAll(n):
    for i in range(1,n):
        qc.h(q[i])  

In [ ]:
def OneStep(K,p):
    global qc,q
    angle1=K[2]*math.pi/p
    qc.cry(angle1,q[1],q[0])
    angle1=K[1]*math.pi/p
    qc.cry(angle1,q[2],q[0])
    angle1=K[0]*math.pi/p
    qc.cry(angle1,q[3],q[0])
    angle1 = K[3] * math.pi/p
    qc.ry(angle1, q[0])
def OneStepMinus(K,p):
    global qc,q
    angle1=-K[2]*math.pi/p
    qc.cry(angle1,q[1],q[0])
    angle1=-K[1]*math.pi/p
    qc.cry(angle1,q[2],q[0])
    angle1=-K[0]*math.pi/p
    qc.cry(angle1,q[3],q[0])
    angle1 = -K[3] * math.pi/p
    qc.ry(angle1, q[0])

In [ ]:
def iterationAlice(s):
    global n, p,K,key, qc, index
    hadamardForAll(n)
    qc.barrier()
    for bit in s:
        if bit =='1':
             OneStep(K,p)
def iterationBob(s):
    global n, p,K,key, qc, index
    for bit in s:
        if bit =='1':
             OneStepMinus(K,p)
    qc.barrier()
    hadamardForAll(n)
    qc.barrier()
    for i in range(n):
        qc.measure(q[i],c[i]) 

In [ ]:
print("Start")
shotss=100
results = []
all_counts = {}
number_of_repetition = 2*p

# Шаг 1: Подготовка и описание схемы
q = QuantumRegister(n)
c = ClassicalRegister(n)
qc = QuantumCircuit(q,c)
iterationAlice("101000")
iterationBob("101100")

# Шаг 2: Выбираем Backend, на котором выполняется схема
backend = FakeCairoV2() # Create a simulated backend

# Шаг 3: Разложение многокубитных гейтов в базисные гейты (зависит от Backend)
transpiled_circuit = transpile(qc, backend=backend, optimization_level=3)

# Шаг 4: Запуск схемы
job = backend.run(transpiled_circuit, shots=shotss)  # Run the circuit with 1024 shots

# Шаг 5: Получаем результат и выводим
result = job.result()
counts = result.get_counts(transpiled_circuit)
try:
    all_counts[number_of_repetition] = counts[key]
except KeyError:
    all_counts[number_of_repetition] = 0

print("all_counts[",number_of_repetition, "]",all_counts[number_of_repetition])
if (all_counts[number_of_repetition]>20): print("Equal")    
else: print("Error")
   
print(f'Measurement Counts: {all_counts}')
plot_histogram(all_counts)
